# Introduction 

The aim of the project is to preliminary test a clustering procedure on multivariate road traffic time series in order to separate different paths between the days of the week and between different months. To create the multivariate time series two fundamental variables are considered the flow $q(t,x)$ and the density $\rho(t,x)$ . The clustering technique used is k-means with soft-dynamic time warping to compare the series. To see if the clustering technique is able to generalize well traffic dynamics, it is also applied to unseen data (the clustering is train on a specific segment of a road and then tested on a different segment with similar boundary level of flow, density and speed). In order to decide the most suitable number of cluster the silhouette coefficients is computed  as well as a similarity measure between the cluster nearest based on soft-DTW.

In section 2 I present the data used, the detectors considered S60 and S1816, the pre-process strategy implemented to treat rough data and normalize them. In section 3 I explain the soft-DTW similarity measure between time series, the k-means algorithm and the silhouette coefficients used to evaluate the technique. Then in section 4 the result are showed: Both detectors are used for training the centroids and as test set. Finally in section 5 conclusion and outlook are reported.       

# Data and Pre-process  

To test the procedure I used data from Minnesota Department of Transportation.The roads considered are I-35W and I-94 the segment of the roads considered are showed in Figure 1. 



\begin{figure}
    \centering
    \includegraphics{maps.png}
    \caption{Satellite image of I-35 and I-94}
\end{figure}
\emph{Figure 1. Satellite image of I-35 and I-94}




The two detectors considered are S60 on I-35W north direction and S1816 on I-94 east direction. Despite the detectors are in different road they share the same number of lanes 5 and they have the similar boundary level in term of flow, density and speed in the 2013 year. In addition both segment do not present ramps 500 meters before and after the detectors. The flow density and speed measurements are downloaded using http://data.dot.state.mn.us/datatools/ for both detectors all lanes are aggregated together and the 30-minute averages from 01/01/2013 00:00 to 31/12/2013 23:30 of the dimensions is computed (17650 observations). The procedure is then repeated also with 6-minute averages (87600 observations).

S60 detector boundary levels 30-minute averages: 

- $\rho(t,x): MIN=0.37\ \ veh/km \ \ \ \ MAX=91.84\ \ veh/km$

- $q(t,x): MIN=74 \ \ veh/h \ \ \ \ MAX=6074\ \ veh/h$

- $v(t,x): MIN=7.02 \ \ km/h \ \ \ \ MAX=98.65\ \ km/h$

S1816 detector boundary levels 30-minute averages: 

- $\rho(t,x): MIN=0.7216\ \ veh/km \ \ \ \ MAX=90.5739\ \ veh/km$

- $q(t,x): MIN= 112\ \ veh/h \ \ \ \ MAX=7266\ \ veh/h$

- $v(t,x): MIN= 8.509\ \ km/h \ \ \ \ MAX=86.79\ \ km/h$




Due to the long period of time considered, from 01/01/2013 to 31/12/2013 for both detectors are present missing values. To reduce their impact on the normalization procedure and on the clustering algorithm  missing values are replaced with substituted values. Generally missing values are created when detectors are out of order, no measure of flow and density are registered for a certain period. To overcome that an imputation process is performed. If for example data on 14/03/2013 at 9:30 are missing the imputation is done by taking the median of the data on 01/03, 07/03, 21/03 and 28/03 at 9:30. By considering the median of the four observation closest to the missing one with the respect to the temporal shift the possible impact of the public holidays in the imputation procedure is reduced. 

Time series, flow and density, are preprocessed using normalization over all period (17520 observations for 30-minute averages and 87600 observations for 6-minute averages). This scaler is such that each output time series is in the range [0,1]  allowing to have identical scales for time series with originally different scales (veh/h and veh/km):
$$\rho_{norm}(t,x)=\frac{\rho(t,x)-MIN}{MAX-MIN}$$.  
$$q_{norm}(t,x)=\frac{q(t,x)-MIN}{MAX-MIN}$$. 


The MinMaxScaler function  from the Python machine learning library  $\textbf{Scikit-learn}$ \footnote{Pedregosa et.Al Scikit-learn: Machine Learning in Python}   trasform each values of the time series proportionally within the range [0,1] preserving the shape. Density and flow scaled series are not amplitude invariant, they do not have the same standard deviation (reached instead by using standardization). 
By looking forward to the clustering procedure density and flow scaled series do not have the same importance in explaining the variance within the cluster, a single modality could be responsible for a large part of the variance inside a specific cluster.
The inverse_transform method of MinMaxScaler, that Undo the scaling of a data point  according to feature_range $\rho(t,x)=\rho(t,x)_{norm}*{MAX-MIN}+ MIN$ and $q(t,x)=q(t,x)_{norm}*{MAX-MIN}+ MIN$, is used in the centroids representation to have a better comprehension of paths, no more in the [0,1] range.

Despite the $MIN \ \ MAX$ for $\rho(t,x)$ and $q(t,x)$ are defined over the 2013 year, the clustering procedure is applied to the 365 daily multivariate time series with 48 daily observations for 30-minute averages and 240 observations 6-minute averages. The choice of scaling the two varaibles with respect to the entire year time and not with respect to the single 365 daily time series is done to preserve variability with respect to different days of the week. For example I simple assume that the $MAX$ density reached on Sunday or Saturday is really low compared to $MAX$ density reached on working days. 

To create the 365 daily normalized multivariate time series a simple $\textbf{reshape}$ procedure is applied: 

$[ts*n,2] \implies [ts,n,d]$ where $ts=365$ is the number of time series, $n=$ is the number of daily observations (48 or 240 depending on minute averages) and $d=2$ is the dimensionality of each time series (flow and density).  	







# Methodology 

## Soft-Dynamic Time Warping 

After having treated time series and pre-processed them in order to have a multivariate time series for every day of the year, I have to define a strategy in order to compare different series both for assign each series in a cluster and update the centroids in the k-means algorithms. Dynamic Time Warping is a technique to measure similarity between two temporal sequences considering not only the temporal alignment but every binary alignment of the two series. For example a similar traffic condition based on flow and density could be recognized in different hours of the day in two different series. The calculation of the DTW distance involves a dynamic programming algorithm that tries to find the optimum warping path between two series under certain constraints.

Given two multivariate series that corresponds to two different days:

$x \in R^{2 x n}$ and $y \in R^{2 x n}$ valued in$ R^2$ (flow and density). Since in this  case $x$ and $y$ have the same lenght DTW can be computed in $O(n^2)$ time. 

Consider a function in order to compare different point of the two series $(x_i \in R^{2}$ and $y_j \in R^{2}) \ \ d \ \ : R^2 × R^2 \Rightarrow R$, such as $d(x_i, y_j)= (\sum_{i,j=1}^n{(|x_i - y_j|^p})^{1/p}$, where usually $d(x_i, y_j)$ is the quadratic Euclidean distance between two vectors. 

A matrix of similarity is computed: 

$\Delta(x,y) := [d(x_i,y_j)]_{i,j} \in R^{2 x n x n}$

$\Delta(x,y)$ can also be defined as local cost matrix, such a matrix must be created for every pair of series compared.  

The DTW algorithm finds the path that minimizes the alignment between $x$ and $y$ by iteratively stepping through  $\Delta(x,y)$, starting at $[d(x_i,y_j)]_{1,1}$ and finishing at $[d(x_i,y_j)]_{n,n}$, and aggregating the cost \footnote{Sardà-Espinosa Comparing Time-Series Clustering Algorithms in R Using the dtwclust Package}. At each step, the algorithm finds the direction in which the cost increases the least under the chosen constraints. These constraints typically consists in forcing paths to lie close to the diagonal. 

By considering $A_{n,n} \subset \{0, 1\}^{n,n}$ all binary alignment matrices the DTW similarity measure reads as follow: 


\begin{equation}
DTW(x,y)= \min_{A \in A_{n,n} } \langle\,A,\Delta(x,y) \rangle
\end{equation}

This creates a warped “path” between $x$ and $y$  that aligns each point in $x$ to the nearest point in $y$. 

However I can not define dynamic time warping as a distance because does not satisfy the triangular inequality, moreover it is not differentiable everywhere due to the  $\min$ operator.  

Soft-Dynamic Time Warping is a variant of DTW that is differentiable. It use the log-sum-exp formulation \footnote{ Cuturi,Blondel Soft-DTW: a Differentiable Loss Function for Time-Series}:

\begin{equation}
DTW^{\gamma}(x,y)= - \gamma \log \sum_{A \in A_{n,n}} exp(- \frac{\langle\,A,\Delta(x,y) \rangle}{\gamma})
  \ \ where \ \ \gamma  \geq 0
\end{equation}

  

Despite considering all alignments and not just the optimal one, soft-DTW can be computed in quadratic time $O(n^2) $ as DTW, however as DTW soft-DTW does not satisfy the triangular inequality. Soft-DTW is a symmetric similarity measure, it supports multivariate series as DTW, and it can provide differently smoothed results by means of a user-defined parameter $\gamma$. 






The "path" created between  $x$ and $y$ is smoother than the one created with DTW. Soft-DTW depends on a hyper-parameter $\gamma$ that controls the smoothing. As showed in Figure 2 and in equation  DTW corresponds to the limit case when $\gamma$ =0. 


\begin{equation}
DTW^{\gamma}(x,y)=\begin{cases}
                     \min_{A \in A_{n,n} } \langle\,A,\Delta(x,y) \rangle ,\ \  \gamma=0 \\
                     - \gamma \log \sum_{A \in A_{n,n}} exp(- \frac{\langle\,A,\Delta(x,y) \rangle}{\gamma}), \ \ \gamma \geq 0 \\
                     \end{cases}
\end{equation}



By default the $\gamma$ hyperparameter is set to 1. 


\begin{figure}
    \centering
    \includegraphics{softdtw.png}
    \caption{}
\end{figure}
\emph{Figure 2.1 Soft-DTW hyperparameter behaviour }



## k-means algorithm

Partitioning methods, such as k-means and k-medoids use an iterative way to create the clusters by moving data points from one cluster to another, based on a distance measure, starting from an initial partitioning. \footnote{k-medoids clustering is very similar to the k-means clustering algorithm. The major difference between them is that while a cluster is represented with its center in the k-means algorithm, it is represented with the most centrally located data point in a cluster in the k-medoids clustering}.

Stepping into time series clustering, since the procedure has been applied to 30-minute averages and 6-minute averages, even if the number of data points is substantially large (365 multivariate time series with 240 daily observations for 6-minute averages) k-means reamins computationally attractive. The complexity of each iteration of the k-means algorithm performed on 365 time series is $O(k × 365)$. This linear complexity is one of the reasons for the popularity of the k-means clustering algorithms \footnote {Soheily-Khah Generalized k-means based clustering for temporal data under time warp Chapter 3}.

The soft-DTW is used in k-means algorithm to assign the series to the clusters and to upload the centroids of the cluster (centroids in a cluster corresponds to the time multivariate series that minimizes the sum of the similarity measures between that time series and all time series inside the cluster). Given the 365 multivariate time series each of them composed by daily observations (48 with 30-minute averages, 240 with 6-minute averages) for both flow and density dimensions the algorithm work as follow: 


$$
\begin{cases}
\textbf{Algorithm} \ \ k-meansclustering \ \ (T,k) \\
\ \ \ \ Input: T = (t_1, t_2, ..., t_{365})  \\
 \ \ \ \ Input: k \ \ the \ \ number \ \ of \ \ clusters \\
 \ \ \ \ Output: {c_1, c_2, ..., c_k} \ \ (set\ \ of\ \ cluster\ \  bi-dimensional\ \ centroids ) \\
\ \ \ \ p=0 \\
 \ \ \ \ Randomly \ \ choose \ \ k \ \ objects \ \ and \ \ make \ \ them \ \ as \  initial \ \ centroids \ \ ({c_1^{(0)}, c_2^{(0)}, ..., c_k^{(0)}}) \\ 
 \ \ \ \  \textbf{repeat} \\
  \ \ \ \ \ \ \ \  Assign \ \ each \ \ data \ \ point \ \ to \ \ the \ \ cluster \ \ with \ \ the \ \ nearest \ \ centroid \ \ using \ \ soft-DTW \\
  \ \ \ \ \ \ \ \ p=p+1 \\
   \ \ \ \ \ \ \ \ // \ \  \textbf{Centroid update} \\
   \ \ \ \ \ \ \ \ \textbf{for} \ \ j=1 \ \ to \ \ k \ \ \textbf{do} \\
    \ \ \ \ \ \ \ \ \ \ Update \ \ the \ \ centroid \ \ c_j^{(p)} \ \ of \ \ each \ \ cluster \ \ using \ \ soft-DTW \\
    \ \ \ \ \ \ \ \ \textbf{end for} \\ 
     \ \ \ \  \textbf{until} \\ \ \  c_j^{(p)} \approx  c_j^{(p-1)} \ \  \ \ j = 1, 2, ..., k \\
     \ \ \ \  Return \ \ c_1 ,c_2 ,... ,c_k \\
\end{cases}
$$


Sometimes different initializations of the centroids lead to very different final clustering results. To overcome this problem 5 times the k-means algorithm is run with different centroids randomly placed at different initial positions. The final results will be the best output of the 5 times consecutive runs in terms of inertia.

## Number of Optimal Clusters, k

K-means, require number of clusters, k, as clustering parameter. Getting the optimal number of clusters is very significant in the analysis. If,for example, k is too high, each time series starts representing a own cluster.

There is no a unique approach for finding the righ number of cluster, I take in account a clustering quality measure and the soft-DTW similarity measure between nearest centroids.  

Since Soft-DTW is differentiable it could be also used as a function to evaluate the cohesion inside each cluster and the separation with respect to the nearest clusters. The silhouette coefficient is a measure of how similar a time series  is to its own cluster (cohesion) compared to other clusters (separation). The silhouette can be computed with the soft-DTW metric, it takes values in the range [-1, 1] \footnote{Rousseeuw Silhouettes: a Graphical Aid to the Interpretation and Validation of Cluster Analysis pag 53-65}. 

Assume that the time series have been clustered via k-means.
For time series $t_i  \in C_k$ (time series $i$ in the cluster $C_k$ )
$a(t_i)$ is the mean distance between time series $t_i$ and all other time series in the same cluster:

$a(t_i)= \frac{1}{| C_k | -1} \sum_{j \in C_k} DTW^{\gamma}(t_i,t_j)$. 

$b(t_i)$ is defined as  the mean dissimilarity of the time series $t_1  \in C_k$ to  the nearest cluster $C_z$ ( where $C_k \neq C_z$) as the mean of the distance from the time series $t_i$ to all time series $\in C_z$:

$b(t_i) = \min_{k \neq z} \frac{1}{|C_z|}\sum_{j \in C_z} DTW^{\gamma}(t_i,t_j)$. 

Finally the silhouette coefficient for a time series is computed as follow: 

$s(t_i) = \frac{b(t_i)-a(t_i)}{max\{a(t_i),b(t_i)\}} if \ \ |C_k| >1$

The coefficients for every time series is averaged to have a global measure. The Mean Silhouette Coefficient for all time series is computed for the different number of clusters to see how this number afflicts the analysis. 

I consider also the soft-DTW similarity measure between the nearest clusters, by taking in account their centroids and applying equation :

- When $k=2$ becomes $DTW^{\gamma}(C_1,C_2)$ where $C_1$ and $C_2$ are centroids of the clusters (multivariate time series (flow and density).

- When $k=d$ where $d\geq 2$, $DTW^{\gamma}(C_j,C_z)$ is applied to all possible binary combinations of centroids forming a symmetric matrix of dimension $(d x d)$ in which the minimum similarity between two centroids is selected.  

As the number of cluster $k$ increases the nearest clusters become close each other until an asymptotic value of soft-DTW is reached.


The code is implemented with the Python machine learning library for time series $\textbf{tslearn}$ \footnote{Tavernard et.Al Tslearn, A Machine Learning Toolkit for Time Series Data}.  








# Result 

## S60 detector 30-minute averages 

### K=2 

S60paths of the centroids k=2.png 


S60train k=2.png 

S60test k=2.png 





### K=3

S60paths of the centroids k=3.png

S60train k=3.png

from 11/03 to 15/03 in blue and from 26/08 to 30/08 in red  
S60 march vs august.png 

from 11/03 to 15/03 in blue and from 09/12 to 13/12 in red 
S60 march vs december.png 


S60test k=3.png

from 11/03 to 15/03 in blue and from 26/08 to 30/08 in red 
S1816 march vs august.png

from 11/03 to 15/03 in blue and from 16/12 to 20/12 in red
S1816 march vs december.png 






### K=4 

S60paths of the centroids k=4.png

S60train k=4.png 

from 27/08 to 30/08 in magenta and from 17/12 to 20/12 in blue 
S60 august december.png 

test on S1816 only 16 days enter in a cluster   

### K=5 

S60train k=5.png 

one cluster with just 7 days 

S60elbow.png

## S1816 detector 30-minute averages

### K=2 

S1816paths of the centroids k=2.png

S1816train k=2.png

S1816test k=2.png 

### K=3 

S1816paths of the centroids k=3.png 

S1816train k=3.png 

from 11/03 to 15/03 in blue and from 09/12 to 13/12 in red 
S1816 march vs december .png

from 11/03 to 15/03 in blue and from 26/08 to 30/08 in red 
S1816 march vs august.png

S1816test K=3.png

### K=4 

S1816paths of the centroids k=4.png

S1816train k=4.png 

from 26/08 to 30/08 in magenta and from 09/12 to 13/12 in blue 
S1816 august vs december flow.png

test on S60 only 10 days in a cluster. 

### K=5 

S1816paths of the centroids k=5.png 

K=0 Sunday 
K=1 December 
K=2 Summer 
K=3 Saturday
K=4 Monday/ Rest of the months. 

S1816train k=5.png

### K >5

S1816train k=6.png 
S1816train k=7.png 

S1816elbow.png 

## S60 detector 6-minute averages 

K=2 and K=3 same result of 30-minute averages

K=4 

S60path of the centroids k=4 6.png
(k=1) and (k=2) really similar 

S60train k=4 6.png 
cluster with just 16 days (k=3) 

## S1816 detector 6-minute averages 


K=5 
S1816paths of the centroids k=5 6.png 
S1816train k=5 6.png 
cluster (k=3) just 11 days.

K=5 30-minute separate 


## Silhouette Coefficients

# Appendix 

## S60 detector misclassification check

05/03/2013 

S60 march.png 

18/03/2013 

S60 march2.png 

11/04/2013 

S60 density April.png

24/08/2013

S60 august.png 

26/10/2013

S60 october.png



## S1816 detector misclassification check

11/02/2013 

S1816 february2.png

18/02/2013 and 22/02/2013 

S1816 february.png 

04/03/2013 and 05/03/2013 

S1816 march .png 

18/03/2013 

S1816 march2.png

04/12/2013

S1816 december .png 